#Importação de Bibliotecas


In [1]:
!pip install moment
!pip install --pre pycaret

import pandas as pd
import numpy as np 
import datetime as dt
from datetime import date
from datetime import datetime
# from pandas_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import plot_confusion_matrix


%matplotlib inline

from sklearn.preprocessing import LabelEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 288 kB 4.0 MB/s 
     |████████████████████████████████| 749 kB 38.2 MB/s 
     |████████████████████████████████| 66 kB 3.7 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2022.6.2
    Uninstalling regex-2022.6.2:
      Successfully uninstalled regex-2022.6.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 487 kB 3.7 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 137 kB 7.2 MB/s 
     |████████████████████████████████| 281 kB 47.4 MB/s 
     |████████████████████████████████| 94 kB 2.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 106 kB 39.1 MB/s 
     |

#Importação da base de dados

In [2]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_excel('/content/drive/Shareddrives/modoulo3-grupo3/Base Colaboradores Everymind_Inteli_2020 a 2022vModelo Preditivo.xlsx', 'Everymind')
df_rec = pd.read_excel('/content/drive/Shareddrives/modoulo3-grupo3/Base Colaboradores Everymind_Inteli_2020 a 2022vModelo Preditivo.xlsx', 'Reconhecimento')

Mounted at /content/drive


#Tratamento de dados

In [3]:
 #Apga o index e seta novamente
def resetIndex(data):
  e = data[data.columns[1]].count() 
  ID = []
  i = 0
  while i < e:
    ID.append(i)
    i += 1

  data['index'] = ID
  data.set_index('index', inplace= True)

#Remove os asentos e caracteries especiais  
df.rename(columns = {'Escolaridade ':'Escolaridade'}, inplace = True)
char = ['Data',' ', 'á', 'ã', 'é', 'ê', 'í', 'ó', 'ú', 'ç', '_de_']
sub_char = ['Dt','_', 'a', 'a', 'e', 'e', 'i', 'o', 'u', 'c', '_']

for i in char:
  e = char.index(i)
  df.columns = df.columns.str.replace(char[e], sub_char[e])
  df_rec.columns = df_rec.columns.str.replace(char[e], sub_char[e])

#Padroniza as data 
today = date.today()
df['Dt_Admissao']= pd.to_datetime(df['Dt_Admissao'])
df['Dt_Admissao'] = df["Dt_Admissao"].dt.strftime("%m-%d-%y")
df['Dt_Admissao']= pd.to_datetime(df['Dt_Admissao'])

df['Dt_Saida']= pd.to_datetime(df['Dt_Saida'])
df['Dt_Saida'] = df["Dt_Saida"].dt.strftime("%m-%d-%y")
df = df.replace(to_replace={'Dt_Saida':{np.nan:today}}, value=None)
df['Dt_Saida']= pd.to_datetime(df['Dt_Saida'])

#Remove espaços e muda PessoaColaboradora para PC
dtg = df.select_dtypes(include = "object").columns
for a in dtg:
  df[a] = df[a].str.replace(' ', '')
  df[a] = df[a].str.replace('PessoaColaboradora', 'PC')

dtg = df_rec.select_dtypes(include = "object").columns
for a in dtg:
  df_rec[a] = df_rec[a].str.replace(' ', '')
  df_rec[a] = df_rec[a].str.replace('PessoaColaboradora', 'PC')

  #Cria a coluna NumeroMeses que calcula o período do funcionario
  def numOfDays(date1, date2):
    return (date2-date1).days

df['NumeroMeses'] = ""


for i in range(0, len(df['Dt_Admissao'])):
  if df['Tipo_Saida'][i] == str('Ativo'):
    df['NumeroMeses'][i] = (((date.today().year - df['Dt_Admissao'][i].year)*12)+date.today().month - df['Dt_Admissao'][i].month)
  else:
    df['NumeroMeses'][i] = (df['Dt_Saida'][i].year - df['Dt_Admissao'][i].year)*12+df['Dt_Saida'][i].month - df['Dt_Admissao'][i].month

df_rec['Numero_Dias'] = ""


for i in range(0, len(df_rec['Dt_Admissao'])):
  df_rec['Numero_Dias'][i] = df_rec['Dt_Vigencia'][i] - df_rec['Dt_Admissao'][i]
  df_rec['Numero_Dias'][i] = df_rec['Numero_Dias'][i].days

df['NumeroMeses'] = df.NumeroMeses.astype(float)

#Cria a coluna situação 
df['Situacao']= np.nan
df = df.replace(to_replace={'Tipo_Saida':{np.nan:'nulo'}}, value=None)

for i in df.index:
    if df['Tipo_Saida'][i]== 'nulo':
        df['Situacao'][i] = 1
    else:
        df['Situacao'][i] = 0

df_rec = pd.get_dummies(df_rec, columns = ['Situacao'])
df_rec = df_rec.drop(['Matricula', 'Situacao_Afastado', 'Situacao_Desligado'],axis=1)
df_rec.rename(columns = {'Situacao_Ativo':'Situacao'}, inplace = True)

#Modifica a coluna de cargos onde todo cargo é representado por um número
# char2 = ['ComercialPl','DevEspecialista','DevJr','DevPl','DevSr','EducaçãoPl','FuncionalEspecialista','FinanceiroJr','FuncionalPl','FuncionalJr','FuncionalPr','FuncionalSr','InfraestruturaJr','MarketingPL','PessoasPl','ScrumMasterJr','TesteJr','TesteSr','ArquitetoSr','Arquiteto','AssistenteII','AssistenteI','AuxiliardeLimpeza','ComercialIS','Consultor','Diretor','Estagiaria','GerenteCSSr','GerentePV','GerenteSr','GerenteCSSr','GerenteVendasIII','GerenteVendasII','GerenteVendasI','Gerente','Trainee-Dev','Trainee-Funcional','VicePresidente']
# sub_char2 = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38']

# for i in char2:
#   e = char2.index(i)
#   df['Cargo'] = df['Cargo'].str.replace(char2[e], sub_char2[e])
df = pd.get_dummies(df, columns = ['Cargo'])
df = pd.get_dummies(df, columns = ['Area'])
df = pd.get_dummies(df, columns = ['Estado_Civil'])
df = pd.get_dummies(df, columns = ['Escolaridade'])
df = pd.get_dummies(df, columns = ['Estado'])
df = pd.get_dummies(df, columns = ['Cidade'])


# #Modifica a coluna Estado_civil
# char3 = ['Solteiro', 'Casado', 'Divorciado', 'UniãoEstável','Separado']
# sub_char3 = ['1', '2', '3', '4','5']

# for i in char3:
#   e = char3.index(i)
#   df['Estado_Civil'] = df['Estado_Civil'].str.replace(char3[e], sub_char3[e])

# #Modifica a coluna Escolaridade 
# char4 = ['PósGraduação', 'Graduação', 'Mestrado', 'Superiorincompleto','EnsinoMédioIncompleto','EnsinoMédio','Técnico']
# sub_char4 = ['1', '2', '3', '4','5','6','7']
# for i in char4:
#   e = char4.index(i)
#   df['Escolaridade'] = df['Escolaridade'].str.replace(char4[e], sub_char4[e])

#Define Masculino como 1 e feminino como 0 
df.Genero[df.Genero == 'Masculino'] = 1
df.Genero[df.Genero == 'Feminino'] = 0

#Define as colunas para tipo float
#df['Cargo'] = df.Cargo.astype(float)
df['Genero'] = df.Genero.astype(float)
# df['Estado_Civil'] = df.Estado_Civil.astype(float)
# df['Escolaridade'] = df.Escolaridade.astype(float)

#Remove algumas colunas
df = df.drop(['Matricula','Dt_Nascimento', 'Dt_Saida', 'Etnia'],axis=1)

#Remove colunas e organiza a tabela de roconhecimento
df_mod = df_rec
df_mod = df_mod.drop(['Novo_Cargo','Novo_Salario', 'Motivo', 'Alterou_Funcao'],axis=1)
df_mod = df_mod.sort_values( by="Codinome", ascending=True)

#Remove a duplicação da tabela de reconhecimento 
df_alt = df_mod.drop_duplicates(subset='Codinome', keep = 'first')
df_dupl = df_mod.loc[df_mod.duplicated(subset='Codinome')]
resetIndex(df_alt)

#Remove da tabela informações de pessoas dispensadas sem justa causa ou com recisão de contrato
n = 0
for i in df['Tipo_Saida']:
  if i == 'DispensasemJustaCausa' or i == 'RescisaoContratoExp-Dispensa':
    df = df.drop(n)
    resetIndex(df)
  else:
    n += 1
    
#Cria uma coluna para todas as promoções 
df_alt['@1'], df_alt['@2'], df_alt['@3'], df_alt['@4'], df_alt['@5'], df_alt['@6'], df_alt['Numero_Promocoes'], df_alt['Tempo_Medio_Promocoes'] = ['0', '0', '0', '0', '0', '0', '0', '0']
df_alt.columns = df_alt.columns.str.replace('@', 'Promocao_')

#Remove as linhas da tabela de reconhecimento onde a pessoa se repete e define em uma meas linhas as promoção das pessoas
for i in df_alt.index:
  e = df_alt['Codinome'][i]
  df_temp = df_mod.query("Codinome in @e").sort_values( by="Numero_Dias", ascending=True)
  resetIndex(df_temp)
  # print(df_temp)
  for e in df_temp.index:
    if e == 0:
      df_alt['Promocao_1'][i] = df_temp['Numero_Dias'][e]
      df_alt['Numero_Promocoes'][i] = 1
    elif e == 1:
      df_alt['Promocao_2'][i] = df_temp['Numero_Dias'][e]
      df_alt['Numero_Promocoes'][i] = 2
    elif e == 2:
      df_alt['Promocao_3'][i] = df_temp['Numero_Dias'][e]
      df_alt['Numero_Promocoes'][i] = 3
    elif e == 3:
      df_alt['Promocao_4'][i] = df_temp['Numero_Dias'][e]
      df_alt['Numero_Promocoes'][i] = 4
    elif e == 4:
      df_alt['Promocao_5'][i] = df_temp['Numero_Dias'][e]
      df_alt['Numero_Promocoes'][i] = 5
    elif e == 5:
      df_alt['Promocao_6'][i] = df_temp['Numero_Dias'][e]
      df_alt['Numero_Promocoes'][i] = 6

df_alt.drop(['Numero_Dias', 'Dt_Vigencia'], axis=1, inplace=True)

#Calcula a media de tempo para todos os funcionarios receberem a priemira promoção 
df_alt_mean = '{:.1f}'.format(df_alt['Promocao_1'].mean())
df_alt_mean

#Calcula a media de tempo para promoção de cada funcionario 
for i in df_alt.index:
  if df_alt['Numero_Promocoes'][i] == 1:
    df_alt['Tempo_Medio_Promocoes'][i] = int(df_alt['Promocao_1'][i]/df_alt['Numero_Promocoes'][i])
  elif df_alt['Numero_Promocoes'][i] == 2:
    df_alt['Tempo_Medio_Promocoes'][i] = int(df_alt['Promocao_2'][i]/df_alt['Numero_Promocoes'][i])
  elif df_alt['Numero_Promocoes'][i] == 3:
    df_alt['Tempo_Medio_Promocoes'][i] = int(df_alt['Promocao_3'][i]/df_alt['Numero_Promocoes'][i])
  elif df_alt['Numero_Promocoes'][i] == 4:
    df_alt['Tempo_Medio_Promocoes'][i] = int(df_alt['Promocao_4'][i]/df_alt['Numero_Promocoes'][i])
  elif df_alt['Numero_Promocoes'][i] == 5:
    df_alt['Tempo_Medio_Promocoes'][i] = int(df_alt['Promocao_5'][i]/df_alt['Numero_Promocoes'][i])
  elif df_alt['Numero_Promocoes'][i] == 6:
    df_alt['Tempo_Medio_Promocoes'][i] = int(df_alt['Promocao_6'][i]/df_alt['Numero_Promocoes'][i])

#Renomeia Codinome para None_Completo
df_alt.rename(columns = {'Codinome':'Nome_Completo'}, inplace = True)
df = df.sort_values( by="Nome_Completo", ascending=True)
resetIndex(df)

#Faz a integração entre a tabela principal com a de reconhecimento 
integrated_dfs = pd.merge(df, df_alt,how="left", on= ["Nome_Completo", "Dt_Admissao", "Situacao"])

#Define as informações nulas para 0 
integrated_dfs = integrated_dfs.fillna(0)

#Cria uma coluna para a primiera promoção
integrated_dfs['Media_Prmc_1'] = df_alt_mean



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to b

#Mostra a tabela final

In [4]:
integrated_dfs

,Nome_Completo,Dt_Admissao,Tipo_Saida,Salario_Mes,Genero,NumeroMeses,Situacao,Cargo_Arquiteto,Cargo_ArquitetoSr,Cargo_AssistenteI,...,Cidade_Vinhedo,Promocao_1,Promocao_2,Promocao_3,Promocao_4,Promocao_5,Promocao_6,Numero_Promocoes,Tempo_Medio_Promocoes,Media_Prmc_1
0,PC1,2022-06-06,nulo,2250.0,1.0,3.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,488.2
1,PC10,2022-02-14,nulo,1600.0,0.0,7.0,1.0,0,0,0,...,0,137,0,0,0,0,0,1,137,488.2
2,PC100,2022-02-03,nulo,4000.0,0.0,7.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,488.2
3,PC101,2019-02-12,nulo,4957.2,1.0,43.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,488.2
4,PC102,2021-04-10,nulo,8715.9,1.0,17.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,488.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,PC95,2022-04-07,nulo,1017.0,1.0,5.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,488.2
430,PC96,2022-03-16,nulo,1017.0,1.0,6.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,488.2
431,PC97,2022-04-07,nulo,1017.0,1.0,5.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,488.2
432,PC98,2020-04-13,nulo,4681.8,1.0,29.0,1.0,0,0,0,...,0,444,0,0,0,0,0,1,444,488.2


In [5]:
integrated_dfs.describe()

,Salario_Mes,Genero,NumeroMeses,Situacao,Cargo_Arquiteto,Cargo_ArquitetoSr,Cargo_AssistenteI,Cargo_AssistenteII,Cargo_AuxiliardeLimpeza,Cargo_ComercialIS,...,Cidade_SãoPaulo,Cidade_SãoVicente,Cidade_TaboãodaSerra,Cidade_Taubaté,Cidade_Uberlândia,Cidade_Unaí,Cidade_Vinhedo,Promocao_1,Numero_Promocoes,Tempo_Medio_Promocoes
count,434.000000,434.000000,434.000000,434.000000,434.000000,434.000000,434.000000,434.000000,434.000000,434.000000,...,434.000000,434.000000,434.000000,434.000000,434.000000,434.000000,434.000000,434.000000,434.000000,434.000000
mean,3372.475611,0.758065,15.751152,0.654378,0.018433,0.006912,0.002304,0.002304,0.002304,0.002304,...,0.382488,0.006912,0.009217,0.002304,0.009217,0.002304,0.002304,106.935484,0.414747,82.324885
std,2501.900487,0.428750,19.534604,0.476120,0.134667,0.082949,0.048002,0.048002,0.048002,0.048002,...,0.486556,0.082949,0.095670,0.048002,0.095670,0.048002,0.048002,343.374082,0.993449,276.379568
min,500.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1500.000000,1.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2750.000000,1.000000,8.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4595.957500,1.000000,20.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,14282.245000,1.000000,199.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4262.000000,6.000000,4262.000000


#Testes dos modelos

#Passo 1: estimar com base na coluna 'Value'


### Precisamos dividir algumas coisas:

1. Precisamos dividir o que é característica (x) do que é o valor a ser predito (y)
2. Também precisamos dividir o que vai ser usado para treinar o modelo e o que vai ser usado para verificar a performance do modelo.

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Dividindo x e y
x = integrated_dfs[['Salario_Mes', 'Genero',
       'NumeroMeses',
       'Cargo_Arquiteto', 'Cargo_ArquitetoSr', 'Cargo_AssistenteI',
       'Cargo_AssistenteII', 'Cargo_AuxiliardeLimpeza', 'Cargo_ComercialIS',
       'Cargo_ComercialPl', 'Cargo_Consultor', 'Cargo_DevEspecialista',
       'Cargo_DevJr', 'Cargo_DevPl', 'Cargo_DevSr', 'Cargo_Diretor',
       'Cargo_EducaçãoPl', 'Cargo_Estagiaria', 'Cargo_FinanceiroJr',
       'Cargo_FuncionalEspecialista', 'Cargo_FuncionalJr', 'Cargo_FuncionalPl',
       'Cargo_FuncionalSr', 'Cargo_Gerente', 'Cargo_GerenteCSSr',
       'Cargo_GerentePV', 'Cargo_GerenteSr', 'Cargo_GerenteVendasI',
       'Cargo_GerenteVendasII', 'Cargo_GerenteVendasIII',
       'Cargo_InfraestruturaJr', 'Cargo_MarketingPL', 'Cargo_PessoasPl',
       'Cargo_ScrumMasterJr', 'Cargo_TesteJr', 'Cargo_TesteSr',
       'Cargo_Trainee-Dev', 'Cargo_Trainee-Funcional', 'Cargo_VicePresidente',
       'Area_AMS', 'Area_AgenciaDigital', 'Area_Analytics', 'Area_BAC',
       'Area_BPM', 'Area_BestMinds', 'Area_CPG&Retail', 'Area_CPG&RetailI',
       'Area_CPG&RetailII', 'Area_Commerce', 'Area_Core&Industrias',
       'Area_Core&IndustriasI', 'Area_Core&IndustriasII', 'Area_Diretoria',
       'Area_Education', 'Area_Financeiro', 'Area_Infraestrutura',
       'Area_Integration', 'Area_MktCloud', 'Area_PS', 'Area_People',
       'Area_Produtos', 'Area_Vendas', 'Promocao_1', 'Promocao_2',
       'Promocao_3', 'Promocao_4', 'Promocao_5', 'Promocao_6',
       'Numero_Promocoes', 'Tempo_Medio_Promocoes', 'Media_Prmc_1',
       'Estado_Civil_Casado', 'Estado_Civil_Divorciado',
       'Estado_Civil_Separado', 'Estado_Civil_Solteiro',
       'Estado_Civil_UniãoEstável', 'Escolaridade_EnsinoMédio',
       'Escolaridade_EnsinoMédioIncompleto', 'Escolaridade_Graduação',
       'Escolaridade_Mestrado', 'Escolaridade_PósGraduação',
       'Escolaridade_Superiorincompleto', 'Escolaridade_Técnico', 'Estado_BA',
       'Estado_CE', 'Estado_DF', 'Estado_GO', 'Estado_MA', 'Estado_MG',
       'Estado_MS', 'Estado_PA', 'Estado_PB', 'Estado_PE', 'Estado_PR',
       'Estado_RJ', 'Estado_RN', 'Estado_RS', 'Estado_SC', 'Estado_SE',
       'Estado_SP']]



### Balanceamento dos dados com Over Sampling

In [7]:
from imblearn.over_sampling import RandomOverSampler

#Separação das variáveis x e y para preparar o balanceamento
x = scaler.fit_transform(x)
y = integrated_dfs[['Situacao']]

#Geração das variáveis com dados balanceados
ros = RandomOverSampler(random_state=0)
x_resampled, y_resampled = ros.fit_resample(x, y)

In [8]:
print(x)

[[0.12697496 1.         0.02487562 ... 0.         0.         0.        ]
 [0.07981283 0.         0.04477612 ... 0.         0.         0.        ]
 [0.25394992 0.         0.04477612 ... 0.         0.         1.        ]
 ...
 [0.03751203 1.         0.03482587 ... 0.         0.         1.        ]
 [0.30341936 1.         0.15422886 ... 0.         0.         1.        ]
 [0.03751203 1.         0.04477612 ... 1.         0.         0.        ]]


###PCA

In [9]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=5)
pca_x = pca.fit_transform(x_resampled)

In [10]:
pca.explained_variance_ratio_

array([0.13705073, 0.10714834, 0.06616714, 0.06314995, 0.05615078])

In [11]:
# Dividindo dados para treino e dados para teste
x_train, x_test, y_train, y_test = train_test_split(pca_x, y_resampled, 
                                                    test_size = 0.2, 
                                                    random_state = 1) # qualquer valor como semente do pseudo-rando

In [12]:
x_train.shape

(454, 5)

In [13]:
y_train.shape

(454, 1)

#Normalizar Dados

In [14]:
from sklearn.preprocessing import RobustScaler, StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [15]:
df_z_scaled = integrated_dfs.copy() 
for column in df_z_scaled.columns: 
    df_z_scaled[['Salario_Mes', 'NumeroMeses','Tempo_Medio_Promocoes','Numero_Promocoes']] = (df_z_scaled[['Salario_Mes', 'NumeroMeses','Tempo_Medio_Promocoes','Numero_Promocoes']] -
                           df_z_scaled[['Salario_Mes', 'NumeroMeses','Tempo_Medio_Promocoes','Numero_Promocoes']].mean()) / df_z_scaled[['Salario_Mes', 'NumeroMeses','Tempo_Medio_Promocoes','Numero_Promocoes']].std()     
display(df_z_scaled)

,Nome_Completo,Dt_Admissao,Tipo_Saida,Salario_Mes,Genero,NumeroMeses,Situacao,Cargo_Arquiteto,Cargo_ArquitetoSr,Cargo_AssistenteI,...,Cidade_Vinhedo,Promocao_1,Promocao_2,Promocao_3,Promocao_4,Promocao_5,Promocao_6,Numero_Promocoes,Tempo_Medio_Promocoes,Media_Prmc_1
0,PC1,2022-06-06,nulo,-0.448649,1.0,-0.652747,1.0,0,0,0,...,0,0,0,0,0,0,0,-0.417481,-0.297869,488.2
1,PC10,2022-02-14,nulo,-0.708452,0.0,-0.447982,1.0,0,0,0,...,0,137,0,0,0,0,0,0.589113,0.197826,488.2
2,PC100,2022-02-03,nulo,0.250819,0.0,-0.447982,1.0,0,0,0,...,0,0,0,0,0,0,0,-0.417481,-0.297869,488.2
3,PC101,2019-02-12,nulo,0.633408,1.0,1.394901,1.0,0,0,0,...,0,0,0,0,0,0,0,-0.417481,-0.297869,488.2
4,PC102,2021-04-10,nulo,2.135746,1.0,0.063930,1.0,0,0,0,...,0,0,0,0,0,0,0,-0.417481,-0.297869,488.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,PC95,2022-04-07,nulo,-0.941475,1.0,-0.550364,1.0,0,0,0,...,0,0,0,0,0,0,0,-0.417481,-0.297869,488.2
430,PC96,2022-03-16,nulo,-0.941475,1.0,-0.499173,1.0,0,0,0,...,0,0,0,0,0,0,0,-0.417481,-0.297869,488.2
431,PC97,2022-04-07,nulo,-0.941475,1.0,-0.550364,1.0,0,0,0,...,0,0,0,0,0,0,0,-0.417481,-0.297869,488.2
432,PC98,2020-04-13,nulo,0.523332,1.0,0.678225,1.0,0,0,0,...,0,444,0,0,0,0,0,0.589113,1.308617,488.2


## Visualizar a distribuição das classes (Plotly)

In [16]:
import plotly.express as px

fig = px.scatter(df, x="Salario_Mes", y="NumeroMeses", color="Situacao",
                 hover_data=['Genero'])
fig.show()

## Regressao Logistica

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Criando o modelo
model = LogisticRegression()
model.fit(x_train, y_train)

# Predizendo
y_pred = model.predict(x_test)

print('Accuracy: ',metrics.accuracy_score(y_test, y_pred))


Accuracy:  0.6491228070175439


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



## KNN (K Nearest Neighbor)


A ideia principal do KNN é determinar o rótulo de classificação de uma amostra baseado nas amostras vizinhas advindas de um conjunto de treinamento.


In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix

# Instaciação do obj Algoritmo
knn = KNeighborsClassifier(n_neighbors=7)
# Treino # x = Features, y = Label/Target
knn.fit( x_train, y_train.squeeze() ) # squeeze() -> df para series


# Teste de Acuracidade (accuracy)
print('Acuracidade (treino): ', knn.score( x_train, y_train ))
print('Acuracidade (teste): ', knn.score( x_test, y_test ))


# Matriz de confusão do modelo Random Forest
#plot_confusion_matrix(knn, x_test, y_test, cmap='BuGn')

Acuracidade (treino):  0.7709251101321586
Acuracidade (teste):  0.6929824561403509


In [19]:
# realizando predições com o conjunto de teste
y_pred = knn.predict(x_test)

Probabilidade da predição por classe e por amostra do x_test

In [20]:
# probabilidade de cada classe
y_prob = knn.predict_proba(x_test)

## Decision Tree

As Árvores de Decisão são um método de aprendizado supervisionado não paramétrico usado para classificação e regressão. O objetivo é criar um modelo que preveja o valor de uma variável de destino aprendendo regras de decisão simples inferidas a partir dos recursos de dados

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import plot_confusion_matrix

# Criando o modelo Arvore de Decisão
dt = DecisionTreeClassifier(max_depth=3) 

# Treinando e rodando o modelo
dt.fit( x_train, y_train.squeeze() ) 

# Teste de Acuracidade (accuracy)
print('Acuracidade (treino): ', dt.score( x_train, y_train ))
print('Acuracidade (teste): ', dt.score( x_test, y_test ))

# Matriz de confusão do modelo Random Forest
#plot_confusion_matrix(dt, x_test, y_test, cmap='Oranges')

Acuracidade (treino):  0.7136563876651982
Acuracidade (teste):  0.6491228070175439


##REPRESENTACAO

In [22]:
from sklearn.tree import export_graphviz 
import graphviz


# Composição e montagem Arvore
dot_data = export_graphviz(dt, out_file=None, class_names=list(map(str, dt.classes_))) 
graph = graphviz.Source(dot_data)
graph.render('Arvore.pdf')


'Arvore.pdf.pdf'

## Random Forest

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix

#Treino e teste do modelo Random Forest
modelo = RandomForestClassifier(max_depth=4)
modelo.fit( x_train, y_train.squeeze() )


#Métrica de acurácia do modelo Random Forest
print('Acc treino: ', modelo.score(x_train, y_train ))
print('Acc teste: ', modelo.score(x_test, y_test.squeeze() ))

#Matriz de confusão do modelo Random Forest
#plot_confusion_matrix(modelo, x_test, y_test, cmap='Greens')

Acc treino:  0.7995594713656388
Acc teste:  0.7017543859649122


## SVM


In [24]:
from sklearn import svm


#Treino e teste do modelo SVM (Support Vector Machines)
clf = svm.SVC()
clf.fit(x_train, y_train)

#Métrica de acurácia do modelo SVM
print('Acuracidade (treino): ', clf.score(x_train, y_train))
print('Acuracidade (teste): ', clf.score(x_test, y_test))

#Matriz de confusão do modelo SVM
#plot_confusion_matrix(clf, x_test, y_test, cmap='Blues')

Acuracidade (treino):  0.7290748898678414
Acuracidade (teste):  0.6578947368421053


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



##Naive bayes predictive model

In [25]:
from sklearn.naive_bayes import GaussianNB

# Treino/teste Implementação naive Bayes
nb = GaussianNB()
nb.fit(x_train, y_train)

# Métrica de acurácia do modelo Naive Bayes
print("Naive Bayes score: ",nb.score(x_test, y_test))
print('Acuracidade (treino): ', nb.score(x_train, y_train))



# Matriz de confusão do modelo Naive Bayes
#plot_confusion_matrix(nb, x_test, y_test, cmap='YlOrRd')

Naive Bayes score:  0.631578947368421
Acuracidade (treino):  0.6850220264317181


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



## XGBoost

In [26]:
from sklearn.ensemble import GradientBoostingClassifier

# Criando modelo XGBoost
xgb = GradientBoostingClassifier()

# Treinando e rodando o modelo
xgb.fit(x_train, y_train)
y_pred = xgb.predict(x_test)

# Metrica e resultado do modelo XGBoost
print('Acc treino', xgb.score(x_train, y_train))
print('Acc teste', xgb.score(x_test, y_test.squeeze()))


# Matriz de confusão do modelo XGBoost
#plot_confusion_matrix(xgb, x_test, y_test, cmap='Reds')

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



Acc treino 0.9295154185022027
Acc teste 0.8333333333333334


## AdaBoost

In [27]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import recall_score, precision_score, f1_score
# Criando o modelo AdaBoost
model = AdaBoostClassifier()

# Treinando e rodando o modelo
model.fit(x_train, y_train.squeeze())
y_pred = model.predict(x_test)

# Metrica e resultado do modelo XGBoost
print('Acc treino: ', model.score(x_train, y_train ))
print('Acc teste: ', model.score(x_test, y_test.squeeze() ))

# Matriz de confusão do modelo AdaBoost
#plot_confusion_matrix(model, x_test, y_test, cmap='Purples')

Acc treino:  0.8237885462555066
Acc teste:  0.6578947368421053


In [28]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

#ADABOOST com Hiperparâmetros

In [29]:
# Definir o espaço possível dos hiperparametros a serem testados no modelo
parametros = {'n_estimators': [30,35,45,50],
              'learning_rate': [1.2,1.3,1.35,1.45],
              'algorithm': ['SAMME', 'SAMME.R']}
             

# Busca de grade, define os melhores parametros a serem usados
grid_search = GridSearchCV(estimator = AdaBoostClassifier(), param_grid=parametros)
grid_search.fit(x_train, y_train.squeeze())

# Printa os melhores parametros e a melhor nota entre eles
print(grid_search.best_score_)
print(grid_search.best_params_)

# Cria uma nova variavel utilizando os novos hiperparametros
model2 = grid_search.best_estimator_
model2
best_model2 = AdaBoostClassifier(grid_search.best_params_)
print(best_model2)

0.7136019536019538
{'algorithm': 'SAMME.R', 'learning_rate': 1.35, 'n_estimators': 50}
AdaBoostClassifier(base_estimator={'algorithm': 'SAMME.R',
                                   'learning_rate': 1.35, 'n_estimators': 50})


In [30]:
y_pred = model2.predict(x_test)
print('Acc treino: ', model2.score(x_train, y_train ))
print('Acc teste: ', model2.score(x_test, y_test.squeeze() ))
print( 'Revocação: ', recall_score( y_test, y_pred ))
print( 'Precisão: ', precision_score( y_test, y_pred ))
print( 'F1_score: ', f1_score( y_test, y_pred ))

Acc treino:  0.8590308370044053
Acc teste:  0.6929824561403509
Revocação:  0.6071428571428571
Precisão:  0.723404255319149
F1_score:  0.6601941747572816


#XGBOOST com Hiperparâmetros


In [31]:
# Definir o espaço possível dos hiperparametros a serem testados no modelo
parametros = {'loss': ['exponential'],
              'learning_rate': [0.2,0.3],
              'n_estimators': [100],
              'subsample': [1.0],
              'criterion':['friedman_mse'],
              'max_depth': [3,4,5],
              'init': ['zero'],
              'max_features': [None]}

# Busca de grade, define os melhores parametros a serem usados
grid_search = GridSearchCV(estimator = GradientBoostingClassifier(), param_grid=parametros)
grid_search.fit(x_train, y_train.squeeze())

# Printa os melhores parametros e a melhor nota entre eles
print(grid_search.best_score_)
print(grid_search.best_params_)

# Cria uma nova variavel utilizando os novos hiperparametros
XGB2 = grid_search.best_estimator_
best_XGB2 = GradientBoostingClassifier(criterion = 'friedman_mse', init = 'zero', learning_rate = 0.2, loss= 'exponential', max_depth = 3, max_features= None, n_estimators= 100, subsample= 1.0)
print(best_XGB2)


0.7753357753357752
{'criterion': 'friedman_mse', 'init': 'zero', 'learning_rate': 0.3, 'loss': 'exponential', 'max_depth': 3, 'max_features': None, 'n_estimators': 100, 'subsample': 1.0}
GradientBoostingClassifier(init='zero', learning_rate=0.2, loss='exponential')


In [32]:
y_pred = XGB2.predict(x_test)
print('Acc treino: ', XGB2.score(x_train, y_train ))
print('Acc teste: ', XGB2.score(x_test, y_test.squeeze() ))
print( 'Revocação: ', recall_score( y_test, y_pred ))
print( 'Precisão: ', precision_score( y_test, y_pred ))
print( 'F1_score: ', f1_score( y_test, y_pred ))

Acc treino:  0.9977973568281938
Acc teste:  0.8333333333333334
Revocação:  0.7678571428571429
Precisão:  0.8775510204081632
F1_score:  0.819047619047619


#RandomForest.random com Hiperparâmetros


In [33]:
# Definir o espaço possível dos hiperparametros a serem testados no modelo
parametros = {'n_estimators': [90,100,100,120,130,140,150],
              'criterion': ['gini', 'entropy'],
              'min_samples_split':[0.5,0.8,1.0,2,3],
              'max_depth': [None],
              'max_features': ['sqrt', None,'log2'],
              'verbose': [0],
              'class_weight':['balanced', 'balanced_subsample']}

# Busca de aleatoria para definir os melhores parametros a serem usados
clf = RandomizedSearchCV(modelo, parametros, n_iter=10, verbose=2)

clf.fit(x_train, y_train.squeeze())

# Printa os melhores parametros e a melhor nota entre eles
print(clf.best_params_)
print(clf.best_score_)
best_modelo = clf.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END class_weight=balanced, criterion=gini, max_depth=None, max_features=None, min_samples_split=0.5, n_estimators=140, verbose=0; total time=   0.2s
[CV] END class_weight=balanced, criterion=gini, max_depth=None, max_features=None, min_samples_split=0.5, n_estimators=140, verbose=0; total time=   0.2s
[CV] END class_weight=balanced, criterion=gini, max_depth=None, max_features=None, min_samples_split=0.5, n_estimators=140, verbose=0; total time=   0.2s
[CV] END class_weight=balanced, criterion=gini, max_depth=None, max_features=None, min_samples_split=0.5, n_estimators=140, verbose=0; total time=   0.2s
[CV] END class_weight=balanced, criterion=gini, max_depth=None, max_features=None, min_samples_split=0.5, n_estimators=140, verbose=0; total time=   0.2s
[CV] END class_weight=balanced_subsample, criterion=entropy, max_depth=None, max_features=sqrt, min_samples_split=3, n_estimators=150, verbose=0; total time=   0.4s
[CV]

In [34]:

y_pred = best_modelo.predict(x_test)
print('Acc treino: ', best_modelo.score(x_train, y_train ))
print('Acc teste: ', best_modelo.score(x_test, y_test.squeeze() ))
print( 'Revocação: ', recall_score( y_test, y_pred ))
print( 'Precisão: ', precision_score( y_test, y_pred ))
print( 'F1_score: ', f1_score( y_test, y_pred ))

Acc treino:  1.0
Acc teste:  0.8245614035087719
Revocação:  0.75
Precisão:  0.875
F1_score:  0.8076923076923077


#RandomForest.grid com Hiperparâmetros


In [35]:
# Definir o espaço possível dos hiperparametros a serem testados no modelo
parametros = {'n_estimators': [120],
              'criterion': ['gini', 'entropy'],
              'max_depth': [None],
              'max_features': ['sqrt', None],
              'verbose': [0],
              'class_weight':['balanced', 'balanced_subsample']}

# Busca de grade, define os melhores parametros a serem usados
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid=parametros)
grid_search.fit(x_train, y_train.squeeze())

# Printa os melhores parametros e a melhor nota entre eles
print(grid_search.best_score_)
print(grid_search.best_params_)

# Cria uma nova variavel utilizando os novos hiperparametros
RF = grid_search.best_estimator_
best_RF = RandomForestClassifier(grid_search.best_params_)
print(best_RF)

0.773113553113553
{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 120, 'verbose': 0}
RandomForestClassifier(n_estimators={'class_weight': 'balanced',
                                     'criterion': 'gini', 'max_depth': None,
                                     'max_features': 'sqrt',
                                     'n_estimators': 120, 'verbose': 0})


In [36]:
y_pred = RF.predict(x_test)
print('Acc treino: ', RF.score(x_train, y_train ))
print('Acc teste: ', RF.score(x_test, y_test.squeeze() ))
print( 'Revocação: ', recall_score( y_test, y_pred ))
print( 'Precisão: ', precision_score( y_test, y_pred ))
print( 'F1_score: ', f1_score( y_test, y_pred ))

Acc treino:  1.0
Acc teste:  0.8333333333333334
Revocação:  0.7857142857142857
Precisão:  0.8627450980392157
F1_score:  0.8224299065420562


# **Naive Bayes com Hiperparâmetros**



In [37]:
# Definir o espaço possível dos hiperparametros a serem testados no modelo
parametros = { 'priors': [None], 
              'var_smoothing': [1e-09]}

# Busca de grade, define os melhores parametros a serem usados
grid_search = GridSearchCV(estimator = GaussianNB(), param_grid=parametros)
grid_search.fit(x_train, y_train.squeeze())

# Printa os melhores parametros e a melhor nota entre eles
print(grid_search.best_score_)
print(grid_search.best_params_)

0.6894505494505495
{'priors': None, 'var_smoothing': 1e-09}


In [38]:
y_pred = nb.predict(x_test)
print('Acc treino: ', nb.score(x_train, y_train ))
print('Acc teste: ', nb.score(x_test, y_test.squeeze() ))
print( 'Revocação: ', recall_score( y_test, y_pred ))
print( 'Precisão: ', precision_score( y_test, y_pred ))
print( 'F1_score: ', f1_score( y_test, y_pred ))

Acc treino:  0.6850220264317181
Acc teste:  0.631578947368421
Revocação:  0.625
Precisão:  0.625
F1_score:  0.625


# **Pycaret**

In [39]:
from pycaret.classification import *
df_pycaret = setup(integrated_dfs, ignore_features = ['Tipo_Saida','Dt_Admissao','Nome_Completo'],target = 'Situacao')

,Description,Value
0,Session id,4013
1,Target,Situacao
2,Target type,Binary
3,Original data shape,"(434, 220)"
4,Transformed data shape,"(434, 193)"
5,Transformed train set shape,"(303, 193)"
6,Transformed test set shape,"(131, 193)"
7,Ignore features,3
8,Numeric features,213
9,Categorical features,6


In [43]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7819,0.8318,0.8632,0.8181,0.8379,0.5041,0.5126,0.4090
ridge,Ridge Classifier,0.7259,0.0000,0.8034,0.7876,0.7927,0.3859,0.3919,0.1040
et,Extra Trees Classifier,0.7194,0.7298,0.8884,0.7390,0.8059,0.3127,0.3290,0.3430
gbc,Gradient Boosting Classifier,0.7097,0.6240,0.8987,0.7260,0.8023,0.2765,0.2972,0.2410
lightgbm,Light Gradient Boosting Machine,0.7065,0.6413,0.8934,0.7235,0.7990,0.2730,0.2952,0.1560
ada,Ada Boost Classifier,0.7063,0.6764,0.8937,0.7247,0.7995,0.2707,0.2907,0.2310
rf,Random Forest Classifier,0.7062,0.7237,0.8934,0.7237,0.7992,0.2721,0.2909,0.3500
dt,Decision Tree Classifier,0.6997,0.6156,0.8884,0.7200,0.7947,0.2547,0.2725,0.1390
lda,Linear Discriminant Analysis,0.6667,0.7261,0.7479,0.7464,0.7443,0.2623,0.2675,0.1510
dummy,Dummy Classifier,0.6535,0.5000,1.0000,0.6535,0.7904,0.0000,0.0000,0.1310


Processing:   0%|          | 0/61 [00:00<?, ?it/s]

In [41]:
predict_model(best).columns


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.8092,0.8390,0.8837,0.8352,0.8588,0.5654,0.5675


Index(['Salario_Mes', 'Genero', 'NumeroMeses', 'Cargo_Arquiteto',
       'Cargo_AssistenteI', 'Cargo_AssistenteII', 'Cargo_ComercialIS',
       'Cargo_ComercialPl', 'Cargo_Consultor', 'Cargo_DevEspecialista',
       ...
       'Promocao_5', 'Promocao_6_0', 'Promocao_6_0#', 'Promocao_6_964',
       'Promocao_6_1292', 'Numero_Promocoes', 'Tempo_Medio_Promocoes',
       'Situacao', 'prediction_label', 'prediction_score'],
      dtype='object', length=195)